In [2]:
#### import packages ####
import os
import ipywidgets as widgets
from ipywidgets import Label, HBox, VBox
from ipyfilechooser import FileChooser

#### import MY packages ####
from parsers import RHSCETparser
from setups import MetalDensity
from setups import utilities
from setups import UIdisplay

style = {"description_width": "initial"}

## UT1. Original Extracted Metal Density Format Exchange ##
1. The original metal density is provided from YW's utility
2. Create layer-mapping file, align the layer name with Totem-EM techfile
3. Translate the original metal density file to Lynden's utility format

### format comparsion ###
Original extracted metal density format:\
llx,lly,urx,ury,layer,metal_density(%)

Lynden's utility format:\
<METAL/VIA/Metal_global/Via_global>,llx,lly,urx,ury,metal_density(%)

In [16]:
### Create CASE folder for power/thermal analysis ###
#CASE = widgets.Text(value="TEST", description="CASE Name:", style=style, disabled=False)

### MDfile format: llx,lly,urx,ury,layer,metal_density(%) ###
MDPath_org = FileChooser("./")   ### power file generated from FC
totemPath = FileChooser("./")
layermapPath = FileChooser("./")

#PWType = widgets.RadioButtons(options=["IPF","CSV","TILE"], description="Power file type", disabled=False)

HB1 = HBox([Label("Select Original Metal Density File:"), MDPath_org])
HB2 = HBox([Label("Select Totem-EM File:"), totemPath])
HB3 = HBox([Label("Select Layer Mapping File:"), layermapPath])
VBox([HB1, HB2, HB3])

In [17]:
import importlib
importlib.reload(MetalDensity)

### Translate to format: layer,llx,lly,urx,ury,Metal_density(%) ###
### default save to: outputFolder="./", outputName="metal_density.setup" ###
MetalDensity.translate2MDSetup(givenMD=MDPath_org.selected, totemPath=totemPath.selected, layermapPath=layermapPath.selected)

C:\Users\chienchi\OneDrive - Intel Corporation\Documents\PyCodes\Thermal\RHSC_ET\DB\tech\fake1P11M_BS4Mi.tech Parsing...
[['BM4'], ['BV3'], ['BM3'], ['BV2'], ['BM2'], ['BV1'], ['BM1'], ['BV0'], ['BM0'], ['TSV'], ['POD', 'NOD'], ['NPTAB'], ['M0'], ['VIA0'], ['M1'], ['VIA1'], ['M2'], ['VIA2'], ['M3'], ['VIA3'], ['M4'], ['VIA4'], ['M5'], ['VIA5'], ['M6'], ['VIA6'], ['M7'], ['VIA7'], ['M8'], ['VIA8'], ['M9'], ['VIA9'], ['M10'], ['VIA10'], ['M11']]


In [45]:
MDPath_forET = FileChooser("./")   ### metal density setup file ###
totemPath = FileChooser("./")
layermapPath = FileChooser("./")

HB1 = HBox([Label("Select Metal Density Setup:"), MDPath_forET])
HB2 = HBox([Label("Select Totem-EM File:"), totemPath])
HB3 = HBox([Label("Select Layer Mapping File:"), layermapPath])

llx = widgets.FloatText(value=0.0, description="LLX", disabled=False)
lly = widgets.FloatText(value=0.0, description="LLY", disabled=False)
urx = widgets.FloatText(value=0.0, description="URX", disabled=False)
ury = widgets.FloatText(value=0.0, description="URY", disabled=False)
HB4 = HBox([llx, lly, urx, ury])
VBox([HB1, HB2, HB3, Label("Design Area:"), HB4])

In [48]:
import importlib
importlib.reload(MetalDensity)

designArea = [llx.value, lly.value, urx.value, ury.value]

### Translate to ET Metal Density format ###
### default save to: outputFolder="./", ETMDName="density.csv" ###
MetalDensity.loadMetalDensitySetup(totemPath=totemPath.selected, MDPath=MDPath_forET.selected, 
                                   layermapPath=layermapPath.selected, area=designArea)

C:\Users\chienchi\OneDrive - Intel Corporation\Documents\PyCodes\Thermal\RHSC_ET\DB\tech\fake1P11M_BS4Mi.tech Parsing...
[['BM4'], ['BV3'], ['BM3'], ['BV2'], ['BM2'], ['BV1'], ['BM1'], ['BV0'], ['BM0'], ['TSV'], ['POD', 'NOD'], ['NPTAB'], ['M0'], ['VIA0'], ['M1'], ['VIA1'], ['M2'], ['VIA2'], ['M3'], ['VIA3'], ['M4'], ['VIA4'], ['M5'], ['VIA5'], ['M6'], ['VIA6'], ['M7'], ['VIA7'], ['M8'], ['VIA8'], ['M9'], ['VIA9'], ['M10'], ['VIA10'], ['M11']]


## UT2. ET Metal Density Setup Tools ##
Add metal density to the specified areas (load power IPF to get the cell coordinates)

In [3]:
### Load the Totem-EM ###

totemPath2_1 = FileChooser("./")
HBox([Label("Select Totem-EM File:"), totemPath2_1])

In [9]:
import importlib
from setups import FakeTotemGen
importlib.reload(FakeTotemGen)

totemTF = None
if os.path.isfile(totemPath2_1.selected):
    totemTF = FakeTotemGen.parseTotemTF(totemPath2_1.selected)
    totemTF.parsing()
    totemTF.makeStacking()

if totemTF:
    print("All layers: {}".format(totemTF.stacking))
else:
    print("<ERROR> Totem-EM is un-defined")

layerStack = ",".join([",".join(l) for l in totemTF.stacking])
layerStack = layerStack.split(",")

C:\Users\chienchi\OneDrive - Intel Corporation\Documents\PyCodes\Thermal\RHSC_ET\DB\tech\fake1P11M_BS4Mi.tech Parsing...
All layers: [['BM4'], ['BV3'], ['BM3'], ['BV2'], ['BM2'], ['BV1'], ['BM1'], ['BV0'], ['BM0'], ['TSV'], ['POD', 'NOD'], ['NPTAB'], ['M0'], ['VIA0'], ['M1'], ['VIA1'], ['M2'], ['VIA2'], ['M3'], ['VIA3'], ['M4'], ['VIA4'], ['M5'], ['VIA5'], ['M6'], ['VIA6'], ['M7'], ['VIA7'], ['M8'], ['VIA8'], ['M9'], ['VIA9'], ['M10'], ['VIA10'], ['M11']]
['BM4', 'BV3', 'BM3', 'BV2', 'BM2', 'BV1', 'BM1', 'BV0', 'BM0', 'TSV', 'POD', 'NOD', 'NPTAB', 'M0', 'VIA0', 'M1', 'VIA1', 'M2', 'VIA2', 'M3', 'VIA3', 'M4', 'VIA4', 'M5', 'VIA5', 'M6', 'VIA6', 'M7', 'VIA7', 'M8', 'VIA8', 'M9', 'VIA9', 'M10', 'VIA10', 'M11']


In [19]:
import importlib
importlib.reload(UIdisplay)

GlobalMDList, LayerMDList = [], []
ui = UIdisplay.setupMetalDensityArea(GlobalMDList, LayerMDList, layerStack)
display(ui)

### Optional: Based on the specified cell area to setup metal density ###

In [12]:
cellIPF = FileChooser("./")
PWType = widgets.RadioButtons(options=["CSV"], description="Power file type", disabled=False)

HBox([Label("Select Power File"), cellIPF, PWType])

In [26]:
from setups import FCpowermap

import importlib
importlib.reload(FCpowermap)
importlib.reload(UIdisplay)

### setup cell metal density ###
cellPWRView = FCpowermap.FCPowerView(cellIPF.selected, PWType.value, DesignArea=[])

#print(cellPWRView.cellDict.keys())
ui = UIdisplay.setupMetalDensitybyGivenCellIPF(GlobalMDList, LayerMDList, layerStack, cellList=cellPWRView.cellDict.keys(), FCObject=cellPWRView)
display(ui)

<INFO> Instances Area:26.64,33.08,99.26,105.52; Total PWR=9.302mW


### Generate metal density setup file ###

In [27]:
import importlib
importlib.reload(MetalDensity)

MDSetup = list(set(GlobalMDList)) + list(set(LayerMDList))
MetalDensity.save2MetalDensitySetup(MDList=MDSetup)

In [ ]:
MDSetup2_1 = FileChooser("./")
layermapPath2_1 = FileChooser("./")

HB2_2 = HBox([Label("Select Metal Density Setup:"), MDSetup2_1])
HB2_3 = HBox([Label("Select Layer Mapping File:"), layermapPath2_1])

llx2_1 = widgets.FloatText(value=0.0, description="LLX", disabled=False)
lly2_1 = widgets.FloatText(value=0.0, description="LLY", disabled=False)
urx2_1 = widgets.FloatText(value=0.0, description="URX", disabled=False)
ury2_1 = widgets.FloatText(value=0.0, description="URY", disabled=False)
HB2_4 = HBox([llx2_1, lly2_1, urx2_1, ury2_1])

VBox([HB2_2, HB2_3, Label("Design Area:"), HB2_4])

In [ ]:
import importlib
importlib.reload(MetalDensity)

designArea2_1 = [llx2_1.value, lly2_1.value, urx2_1.value, ury2_1.value]

### Translate to ET Metal Density format ###
### default save to: outputFolder="./", ETMDName="density.csv" ###
MetalDensity.loadMetalDensitySetup(totemPath=totemPath2_1.selected, MDPath=MDSetup2_1.selected, 
                                   layermapPath=layermapPath2_1.selected, area=designArea2_1)

## UT3. ##

In [3]:
root = os.path.join(os.getcwd(), "CASEs")
caseFolder = utilities.createCASE(root, CASE.value)

In [10]:
import importlib
importlib.reload(FCpowermap)

designArea = []
try:
    if isManually.value in ["True"]:
        designArea = [llx.value, lly.value, urx.value, ury.value]
except:
    pass

FCpwrView = FCpowermap.FCPowerView(PWfc.selected, PWType.value, DesignArea=designArea)

#cells = ["i0mfvp803mc1q03x5","i0mfuz443aa1q06x5","i0mfuz803mc1q06x5","i0moai013ab1n09x5","i0moai013ac1d12x5"]
#cells = ["i0mfuz443aa1q06x5"] #["i0maoi302aa1d09x5"] #["i0mfuz443aa1q06x5"]
#FCpwrView.modifyCSV(modifyCells=cells, pwrFactor=50)
#FCpwrView.modifyCSV(modifyCells="ALL", pwrFactor=0.000001)

resolution = widgets.IntText(value=5, description="Tile Resolution(um):", style=style, disabled=False)
topN = widgets.IntText(value=10, description="Top-N High Power Tiles:", style=style, disabled=False)
VBox([resolution, topN])

<INFO> Instances Area:30.49,42.62,102.06,113.08; Total PWR=18.409mW


In [21]:
import importlib
importlib.reload(UIdisplay)

FCpwrView.translate2TilePwr(resolution=resolution.value)
candidatedCells = FCpwrView.TilePwrAnalysis(resolution=resolution.value, topN=topN.value)

### select the candidated cells, droplist: "cell,contributed_power_in_ratio" ###
droplist = []
for c in candidatedCells:
    _str = "{}, {}%".format(c[0], c[2])
    droplist.append(_str)

dumpCells = []
out = UIdisplay.askDropItems2Processing("Select candidated cells", droplist, dumpCells, dropLabel="Cells:")
display(out)

In [12]:
import importlib
importlib.reload(UIdisplay)

XX, YY = 10, 5
grid = UIdisplay.createArrayPanel(XX, YY)
TSize = widgets.IntText(value=5, description="Tile Size(um):", style=style, disabled=False)
VBox([grid, TSize])

In [29]:
grid[2, 4].value

8.0

In [18]:
import importlib
importlib.reload(UIdisplay)

UIdisplay.ArrayPanel2TileIPF(XX, YY, TSize.value, grid)

BBOX: (0.0,0.0)-(100,50), Total Power: 25.502 mW


([0.0, 0.0, 100, 50], './tileIPF.csv')